In [1]:
import pandas as pd
import numpy as np
import os

from env import get_db_url
from acquire import get_db_log_data
from wrangle import wrangle_df

# Acquire:

I need to figure out why these are coming out differently after the join. 
- There's a difference of 52,887 records.
    - It looks like the difference here is due to having 52,893 null values in logs.cohort_id.
    - There are 6 cohorts in the cohorts_id table that do not have any server traffic, so perhaps they didn't exist.
    - 52,887 that didn't have anything to tie them to the cohorts table. Probably cohort ids - come back and verify.
- it appears that there there are some IDs and Cohort IDs that do not much.

## Importing from the database using SQL

In [2]:
df = get_db_log_data()

In [3]:
df.sample(5)

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
232575,2019-04-01,12:22:50,java-ii/exceptions-and-error-handling,325,29.0,97.105.19.58,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2.0
94813,2018-07-31,11:01:43,examples/java/MoviesArray.java,198,24.0,97.105.19.58,24.0,Voyageurs,#voyageurs,2018-05-29,2018-10-11,2018-05-25 22:25:57,2018-05-25 22:25:57,NaN,2.0
214659,2019-03-07,08:48:31,/,344,29.0,97.105.19.58,29.0,Zion,#zion,2019-01-22,2019-06-04,2019-01-20 23:18:57,2019-01-20 23:18:57,NaN,2.0
415036,2019-12-16,07:50:17,1-fundamentals/1.1-intro-to-data-science,363,NaN,4.16.216.161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
336862,2019-08-29,16:12:46,examples/javascript/blog.json,453,33.0,97.105.19.58,33.0,Ceres,#ceres,2019-07-15,2019-12-11,2019-07-15 16:57:21,2019-07-15 16:57:21,NaN,2.0


# Wrangle:

## To-Do:
    - Figure out what created_at and updated_at columns are. 

In [4]:
df = wrangle_df(df)

In [7]:
df.sample(5)

,time,path,user_id,cohort_id,ip,name,start_date,end_date,created_at,updated_at,program_id
date,,,,,,,,,,,
2019-11-20,08:56:16,java-i/introduction-to-java,498,51.0,97.105.19.58,Deimos,2019-09-16,2020-02-27,2019-09-16 13:07:04,2019-09-16 13:07:04,full_stack_java
2020-10-01,17:02:25,java-ii/inheritance-and-polymorphism,732,61.0,72.179.177.9,Bash,2020-07-20,2021-01-21,2020-07-20 19:04:00,2020-09-30 15:54:46,full_stack_java
2020-10-22,14:37:15,mysql/introduction,709,61.0,97.93.179.60,Bash,2020-07-20,2021-01-21,2020-07-20 19:04:00,2020-09-30 15:54:46,full_stack_java
2021-03-11,11:39:10,spring/fundamentals/relationships,753,62.0,72.181.116.86,Jupiter,2020-09-21,2021-03-30,2020-09-21 18:06:27,2020-09-21 18:06:27,full_stack_java
2018-09-24,14:48:30,html-css/css-ii/media-queries,273,26.0,97.105.19.58,Xanadu,2018-09-17,2019-02-08,2018-09-17 19:09:51,2018-09-17 19:09:51,full_stack_java


In [9]:
df['tenure'] = df.end_date - df.start_date

In [16]:
df.columns()

Index(['time', 'path', 'user_id', 'cohort_id', 'ip', 'name', 'start_date',
       'end_date', 'created_at', 'updated_at', 'program_id', 'tenure'],
      dtype='object')

In [14]:
df[df.index < df.end_date & df.index > df.start_date]

TypeError: unsupported operand type(s) for &: 'int' and 'str'

# Questions:

## 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

## 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [5]:
df[df.date > df.end_date].shape[0]

AttributeError: 'DataFrame' object has no attribute 'date'

## 7. Which lessons are least accessed?

## 8. Anything else I should be aware of?